In [1]:
using Laplacians
using ProfileView

In [21]:
n = 100000;
a = wtedChimera(n, 1);

In [22]:
include("../../../src/johnlind.jl")
@time reff = johnlind(a, 0.5);

2580.903470 seconds (13.83 M allocations: 1.272 TB, 6.34% gc time)


In [25]:
n = 20000;
a = wtedChimera(n, 1);

In [26]:
@time reff = johnlind(a, 0.5);

134.896436 seconds (2.77 M allocations: 75.485 GB, 8.05% gc time)


In [18]:
# Profile.clear()
# @profile reff = johnlind(a, 0.5);
# ProfileView.view()

In [23]:
levScore = copy(a);
for i in 1:length(a.nzval)
    levScore.nzval[i] = a.nzval[i] * reff.nzval[i]
end
sum(levScore.nzval) / 2

99992.68098732586

In [17]:
include("../../../src/johnlind.jl")
reffMin = copy(a);
reffMin.nzval = ones(length(a.nzval)) * 2 * n
reffMax = copy(a);
reffMax.nzval = zeros(length(a.nzval))

@time for test in 1:10
    reff = johnlind(a, 0.5)
    for i in 1:length(reff.nzval)
        reffMin.nzval[i] = min(reffMin.nzval[i], reff.nzval[i])
        reffMax.nzval[i] = max(reffMax.nzval[i], reff.nzval[i])
    end
end

ratioMax = 0
for i in 1:length(reff.nzval)
    ratioMax = max(ratioMax, reffMax.nzval[i] / reffMin.nzval[i])
end
println(ratioMax)

149.522583 seconds (9.53 M allocations: 121.897 GB, 7.72% gc time)
1.8281275763348757


In [10]:
inv = pinv(full(lap(a)));
totLevScore = 0
pos = 1
for i in 1:length(a.nzval)
    while a.colptr[pos + 1] <= i
        pos = pos + 1
    end

    p = a.rowval[i]
    q = pos
    w = a.nzval[i]
    
    if p > q
        edge = zeros(a.n)
        edge[p] = 1
        edge[q] = -1
        totLevScore += w * edge' * inv * edge
    end
end
totLevScore

1-element Array{Float64,1}:
 99.0